In [149]:
import sys
print(sys.version)
import re
import pandas as pd

3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [150]:
ar_data = pd.read_json("recipes_raw_nosource_ar.json")
ar_data = ar_data.T
ar_data.dropna(inplace=True)
ar_data['ingredients'] = ar_data['ingredients'].apply(lambda L: " ".join(L))

In [151]:
ar_data['clean_ingredients'] = ar_data['ingredients'].replace(r'[^a-zA-Z\s]', '', regex=True)
ar_data['clean_ingredients'] = ar_data['clean_ingredients'].replace('ADVERTISEMENT', '', regex=True)
ar_data['clean_ingredients'] = ar_data['clean_ingredients'].str.lower()

In [152]:
ar_data['clean_instructions'] = ar_data['instructions'].replace(r'[^a-zA-Z\s]', '', regex=True)
ar_data['clean_instructions'] = ar_data['clean_instructions'].replace(r'\n', ' ', regex=True)
ar_data['clean_instructions'] = ar_data['clean_instructions'].str.lower()

In [155]:
def masktext(text, mask):
    mask_list = mask.split()
    masked_words = [word for word in text.split() if word not in mask_list]
    return " ".join(masked_words)

ar_data['clean_instructions_masked'] = ar_data.apply(lambda l: masktext(l['clean_instructions'], l['clean_ingredients']), axis=1)


In [156]:
ar_data


,title,ingredients,instructions,picture_link,clean_ingredients,clean_instructions,clean_instructions_masked
rmK12Uau.ntP510KeImX506H6Mr6jTu,Slow Cooker Chicken and Dumplings,"4 skinless, boneless chicken breast halves ADV...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S,skinless boneless chicken breast halves tab...,place the chicken butter soup and onion in a s...,place the and in a slow cooker and fill with e...
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,Awesome Slow Cooker Pot Roast,2 (10.75 ounce) cans condensed cream of mushro...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,ounce cans condensed cream of mushroom soup ...,in a slow cooker mix cream of mushroom soup dr...,in a slow cooker and place in slow cooker and ...
clyYQv.CplpwJtjNaFGhx0VilNYqRxu,Brown Sugar Meatloaf,1/2 cup packed brown sugar ADVERTISEMENT 1/2 c...,Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,cup packed brown sugar cup ketchup pound...,preheat oven to degrees f degrees c lightly ...,preheat oven to degrees f degrees c lightly gr...
BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,Best Chocolate Chip Cookies,"1 cup butter, softened ADVERTISEMENT 1 cup whi...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,cup butter softened cup white sugar cup p...,preheat oven to degrees f degrees c cream to...,preheat oven to degrees f degrees c cream toge...
N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,Homemade Mac and Cheese Casserole,8 ounces whole wheat rotini pasta ADVERTISEMEN...,Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm,ounces whole wheat rotini pasta cups fresh ...,preheat oven to degrees f line a quart casser...,preheat oven to degrees f line a quart cassero...
...,...,...,...,...,...,...,...
gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,Thai-Indian Veggie Soup,2 teaspoons olive oil ADVERTISEMENT 1/4 cup mi...,Heat oil in a large pot over medium heat. Add ...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,teaspoons olive oil cup minced fresh ginger...,heat oil in a large pot over medium heat add g...,heat in a large pot over medium heat add cook ...
VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Coconut Milk-Free Panang Curry Chicken,2 cups light cream ADVERTISEMENT 1/4 teaspoon ...,Heat cream and coconut extract in a skillet or...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,cups light cream teaspoon coconut extract o...,heat cream and coconut extract in a skillet or...,heat and in a skillet wok over mediumhigh heat...
Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,Cooked Cold Salad,3 tablespoons bacon grease ADVERTISEMENT 2 cup...,Heat bacon grease in a skillet over medium-hig...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,tablespoons bacon grease cups shredded brus...,heat bacon grease in a skillet over mediumhigh...,heat in a skillet over mediumhigh heat saute a...
ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,Easy Eggnog Creme Brulee,4 egg yolks ADVERTISEMENT 1 tablespoon white s...,Preheat oven to 350 degrees F (175 degrees C)....,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,egg yolks tablespoon white sugar cup eggn...,preheat oven to degrees f degrees c whisk eg...,preheat oven to degrees f degrees c whisk and ...
